<a href="https://colab.research.google.com/github/mathjams/AAAI_2024/blob/main/Specific_Sections.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Looking at Fixations that Lie in Specific Sections

In [ ]:
import matplotlib.pyplot as plt
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, TimeDistributed, Embedding, Concatenate
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.regularizers import l2
from tensorflow.keras.models import Sequential
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.layers import Input, GRU, Dense, TimeDistributed, Embedding, Concatenate
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import MinMaxScaler
import scipy.stats as stats
import statistics

#Data Filtering into Specific Sections

In [ ]:
def filtereddata(urleye, urlhand):
  eye_data=pd.read_excel(urleye)
  hand_data=pd.read_excel(urlhand)
  eye=eye_data.copy()
  hand=hand_data.copy()
  np.array(eye)
  np.array(hand)
  #each area is described as a rectangle with [Xmin, Ymin, Xmax, Ymax]
  backgroundareae=[0, 0.09, 1, 0.462]
  backgroundareah=[0, 0.565, 1, 1]
  referencearea1e=[0, 0.462, 0.208, 0.788]
  referencearea1h=[0, 0.178, 0.208, 0.565]
  facilitationarea1e=[0, 0.788, 0.208, 0.94]
  facilitationarea1h=[0, 0, 0.208, 0.178]
  maintaskareae=[0.208, 0.462, 0.542, 0.94]
  maintaskareah=[0, 0, 0.542, 0.565]
  referencearea2e=[0.542, 0.462, 0.86, 0.94]
  referencearea2h=[0.542, 0, 0.86, 0.565]
  facilitationarea2e=[0.86, 0.462, 1, 0.94]
  facilitationarea2h=[0.86, 0, 1, 0.565]
  badatae=[]
  badatah=[]
  radata1e=[]
  radata1h=[]
  fadata1e=[]
  fadata1h=[]
  madata1e=[]
  madata1h=[]
  radata2e=[]
  radata2h=[]
  fadata2e=[]
  fadata2h=[]
  for i in range(eye.shape[0]):
    if eye.loc[i].x>=backgroundareae[0] and eye.loc[i].x<=backgroundareae[2] and eye.loc[i].y>=backgroundareae[1] and eye.loc[i].y<=backgroundareae[3]:
      badatae.append([eye.loc[i].x, eye.loc[i].y])
    if eye.loc[i].x>=referencearea1e[0] and eye.loc[i].x<=referencearea1e[2] and eye.loc[i].y>=referencearea1e[1] and eye.loc[i].y<=referencearea1e[3]:
      radata1e.append([eye.loc[i].x, eye.loc[i].y])
    if eye.loc[i].x>=facilitationarea1e[0] and eye.loc[i].x<=facilitationarea1e[2] and eye.loc[i].y>=facilitationarea1e[1] and eye.loc[i].y<=facilitationarea1e[3]:
      fadata1e.append([eye.loc[i].x, eye.loc[i].y])
    if eye.loc[i].x>=maintaskareae[0] and eye.loc[i].x<=maintaskareae[2] and eye.loc[i].y>=maintaskareae[1] and eye.loc[i].y<=maintaskareae[3]:
      madata1e.append([eye.loc[i].x, eye.loc[i].y])
    if eye.loc[i].x>=referencearea2e[0] and eye.loc[i].x<=referencearea2e[2] and eye.loc[i].y>=referencearea2e[1] and eye.loc[i].y<=referencearea2e[3]:
      radata2e.append([eye.loc[i].x, eye.loc[i].y])
    if eye.loc[i].x>=facilitationarea2e[0] and eye.loc[i].x<=facilitationarea2e[2] and eye.loc[i].y>=facilitationarea2e[1] and eye.loc[i].y<=facilitationarea2e[3]:
      fadata2e.append([eye.loc[i].x, eye.loc[i].y])
  for i in range(hand.shape[0]):
    if hand.loc[i].x>=backgroundareah[0] and hand.loc[i].x<=backgroundareah[2] and hand.loc[i].y>=backgroundareah[1] and hand.loc[i].y<=backgroundareah[3]:
      badatah.append([hand.loc[i].x, hand.loc[i].y])
    if hand.loc[i].x>=referencearea1h[0] and hand.loc[i].x<=referencearea1h[2] and hand.loc[i].y>=referencearea1h[1] and hand.loc[i].y<=referencearea1h[3]:
      radata1h.append([hand.loc[i].x, hand.loc[i].y])
    if hand.loc[i].x>=facilitationarea1h[0] and hand.loc[i].x<=facilitationarea1h[2] and hand.loc[i].y>=facilitationarea1h[1] and hand.loc[i].y<=referencearea1h[3]:
      fadata1h.append([hand.loc[i].x, hand.loc[i].y])
    if hand.loc[i].x>=maintaskareah[0] and hand.loc[i].x<=maintaskareah[2] and hand.loc[i].y>=maintaskareah[1] and hand.loc[i].y<=maintaskareah[3]:
      madata1h.append([hand.loc[i].x, hand.loc[i].y])
    if hand.loc[i].x>=referencearea2h[0] and hand.loc[i].x<=referencearea2h[2] and hand.loc[i].y>=referencearea2h[1] and hand.loc[i].y<=referencearea2h[3]:
      radata2h.append([hand.loc[i].x, hand.loc[i].y])
    if hand.loc[i].x>=facilitationarea2h[0] and hand.loc[i].x<=facilitationarea2h[2] and hand.loc[i].y>=facilitationarea2h[1] and hand.loc[i].y<=facilitationarea2h[3]:
      fadata2h.append([hand.loc[i].x, hand.loc[i].y])

  return badatae, badatah, radata1e, radata1h, fadata1e, fadata1h, madata1e, madata1h, radata2e, radata2h, fadata2e, fadata2h

In [ ]:
import pandas as pd
import numpy as np

def genfiltereddata(user_type, list):
  resulteye=[]
  resulthand=[]
  maxlen=0
  eye_basic_url='/Users/qyuvks/emily/AAAI/data_set/Eye_'
  hand_basic_url='/Users/qyuvks/emily/AAAI/data_set/Hand_'
  if (user_type=='ASD'):
    numOfUser=9
    eye_basic_url+="ASD_"
    hand_basic_url+='ASD_'
  else:
    eye_basic_url+="TD_"
    hand_basic_url+='TD_'
    numOfUser=17
  for i in range(1, numOfUser+1):
    for j in range(0,2):
      c_eye_url=eye_basic_url+'U'+str(i)+"_Active_"+str(j)+".xlsx"
      c_hand_url=hand_basic_url+'U'+str(i)+"_Active_"+str(j)+".xlsx"
      #asd_eye_data=pd.DataFrame()
      try:
        badatae, badatah, radata1e, radata1h, fadata1e, fadata1h, madata1e, madata1h, radata2e, radata2h, fadata2e, fadata2h = filtereddata(c_eye_url, c_hand_url)
        regions = {1: badatae, 2: badatah, 3: radata1e, 4: radata1h, 5: fadata1e, 6: fadata1h, 7: madata1e, 8: madata1h, 9: radata2e, 10: radata2h, 11: fadata2e, 12: fadata2h}
        lenh=0
        lene=0
        for i in range(len(list)):
          if len(regions[list[i]])!=0 and len(regions[list[i]+1])!=0:
            resulteye.append(regions[list[i]])
            resulthand.append(regions[list[i]+1])
            lene+=len(regions[list[i]])
            lenh+=len(regions[list[i]])
        if lene>maxlen or lenh>maxlen:
          maxlen=max(lene, lenh)
      except IOError:
        print("")
  return resulteye, resulthand, maxlen


#LSTM Model Used

In [ ]:
def LSTM_model4(input_data, output_data, input_max_len, output_max_len, input_features, output_features, latent_dim, number_of_epochs):
    # Pad the input and output sequences to their respective maximum lengths
    input_data_padded = pad_sequences(input_data, maxlen=input_max_len, padding='post', dtype='float32', value=0)
    output_data_padded = pad_sequences(output_data, maxlen=output_max_len, padding='post', dtype='float32', value=0)

    # Shift the output sequences to create the target sequences
    decoder_target_data = np.roll(output_data_padded, shift=-1, axis=1)
    decoder_target_data[:, -1, :] = 0  # Reset last time step to 0 (zero padding)

    # Encoder
    encoder_inputs = Input(shape=(None, input_features), name='encoder_inputs')
    encoder_lstm = LSTM(latent_dim, return_state=True, name='encoder_LSTM')
    encoder_outputs, state_h, state_c = encoder_lstm(encoder_inputs)
    encoder_states = [state_h, state_c]

    # Decoder
    decoder_inputs = Input(shape=(None, output_features), name='decoder_inputs')
    decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True, name='decoder_LSTM')
    decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
    decoder_dense = TimeDistributed(Dense(output_features, activation='linear'))
    decoder_outputs = decoder_dense(decoder_outputs)

    # Define the model that will turn encoder_inputs and decoder_inputs into decoder_outputs
    model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
    model.compile(optimizer=Adam(learning_rate=0.0001), loss='mse')

    # Fit the model
    result = model.fit([input_data_padded, output_data_padded], decoder_target_data,
                       batch_size=50, epochs=number_of_epochs, validation_split=0.2)

    final_loss = result.history['val_loss'][-1]
    print(f'Final validation loss: {final_loss}')

    # Define the encoder model (used for encoding input sequences to their states)
    encoder_model = Model(encoder_inputs, encoder_states)

    # Define the decoder model (used for generating output sequences given the encoded states)
    decoder_state_input_h = Input(shape=(latent_dim,))
    decoder_state_input_c = Input(shape=(latent_dim,))
    decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

    decoder_outputs, state_h_dec, state_c_dec = decoder_lstm(
        decoder_inputs, initial_state=decoder_states_inputs)
    decoder_states = [state_h_dec, state_c_dec]
    decoder_outputs = decoder_dense(decoder_outputs)
    decoder_model = Model(
        [decoder_inputs] + decoder_states_inputs,
        [decoder_outputs] + decoder_states)
  # Extract the number of epochs
    epochs = range(1, number_of_epochs + 1)

# Extract the validation loss from the history
    val_loss = result.history['val_loss']
    return model, encoder_model, decoder_model, val_loss


#Statistical Tests

In [ ]:
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt

#order is meanASD, meanTD, std_asd, std_td, t_statistic, p_value
Statresults = []
X=[]
Ytd=[]
Yasd=[]
# Show plot
areas = {1: ['background', 1], 2: ['main task', 7], 3: ['not background', 3, 5, 7, 9, 11], 4: ['facilitation', 5, 11], 5: ['reference', 3, 9]}
for i in range(1,6):
  numareas = areas[i]
  asdfit=[]
  tdfit=[]
  inputtd, outputtd, max_lentd=genfiltereddata('TD', numareas[1:])
  inputasd, outputasd, max_lenasd=genfiltereddata('ASD', numareas[1:])
  if len(inputasd)>0 and len(inputtd)>0:
    for i in range(3):
      modeltd, encoder_modeltd, decoder_modeltd, val_losstd = LSTM_model4(inputtd, outputtd, max_lentd, max_lentd, 2, 2, 150, 1000)
      modelasd, encoder_modelasd, decoder_modelasd, val_lossasd=LSTM_model4(inputasd, outputasd, max_lenasd, max_lenasd, 2, 2, 150, 1000)
      asdfit.append(min(val_lossasd))
      tdfit.append(min(val_losstd))
  t_statistic, p_value = stats.ttest_ind(asdfit, tdfit)
  meanASD = statistics.mean(asdfit)
  meanTD = statistics.mean(tdfit)
  std_asd = statistics.pstdev(asdfit)
  std_td = statistics.pstdev(tdfit)
  Statresults.append([numareas[0],meanASD, meanTD, std_asd, std_td, t_statistic, p_value])
  X.append(numareas[0])
  Ytd.append(meanTD)
  Yasd.append(meanASD)
# Create DataFrame
dataTD = pd.DataFrame({'x': X, 'y': Ytd})
sns.scatterplot(data=dataTD, x='x', y='y', color='blue')
dataASD = pd.DataFrame({'x': X, 'y': Yasd})
sns.scatterplot(data=dataASD, x='x', y='y', color='red')

# Add labels, title, and legend
plt.xlabel('area')
plt.ylabel('validation error')
plt.title('area vs error')
plt.legend()
plt.show()

print(X, Ytd, Yasd)
print(Statresults)